dropnout soubor
vyexportovat youtube

In [1]:
import os

In [2]:
import pandas as pd

In [3]:
folder = "data_raw"

In [4]:
df = pd.read_csv(os.path.join(folder,"ifpi_raw.csv"))

## Špatně označené roky

In [5]:
df.groupby(['hitparada','obdobi'])['track/album'].nunique().nlargest(50)

hitparada                       obdobi        
SK - SINGLES DIGITAL - TOP 100  8. týden 2017     194
                                5. týden 2022     189
CZ - SINGLES DIGITAL - TOP 100  8. týden 2017     187
CZ - RADIO - TOP 100            8. týden 2017     184
CZ - ALBUMS - TOP 100           8. týden 2017     183
SK - RADIO - TOP 100            8. týden 2017     181
                                5. týden 2022     178
CZ - RADIO - TOP 100            5. týden 2022     177
SK - ALBUMS - TOP 100           8. týden 2017     177
CZ - SINGLES DIGITAL - TOP 100  5. týden 2022     175
CZ - ALBUMS - TOP 100           5. týden 2022     138
SK - ALBUMS - TOP 100           5. týden 2022     134
CZ - RADIO - TOP 100            1. týden 2010     109
CZ - ALBUMS - TOP 100           1. týden 2017     100
                                1. týden 2021     100
                                1. týden 2022     100
                                10. týden 2016    100
                                10.

In [6]:
df.groupby('obdobi')['track/album'].nunique().nlargest(10)

obdobi
8. týden 2017     934
5. týden 2022     839
17. týden 2019    547
20. týden 2019    541
29. týden 2019    541
46. týden 2017    540
28. týden 2019    539
41. týden 2017    538
45. týden 2017    538
20. týden 2017    537
Name: track/album, dtype: int64

In [7]:
df[df['obdobi'] == '8. týden 2017'][['hitparada','soubor']].drop_duplicates()

,hitparada,soubor
35250,CZ - ALBUMS - TOP 100,14-2439.html
40350,CZ - ALBUMS - TOP 100,14-2503.html
132750,CZ - RADIO - TOP 100,2-2439.html
137850,CZ - RADIO - TOP 100,2-2503.html
183881,CZ - RADIO - TOP 20 MODERN ROCK,21-2439.html
184941,CZ - RADIO - TOP 20 MODERN ROCK,21-2503.html
197186,CZ - ALBUMS CLASSICAL - TOP 10,25-2439.html
197712,CZ - ALBUMS CLASSICAL - TOP 10,25-2503.html
215389,CZ - SINGLES DIGITAL - TOP 100,30-2439.html
220489,CZ - SINGLES DIGITAL - TOP 100,30-2503.html


In [8]:
df[df['soubor'].str.contains('2503')].groupby('obdobi').size()

obdobi
8. týden 2017    830
dtype: int64

In [9]:
df[df['obdobi'] == '8. týden 2018']

,Unnamed: 0,hitparada,obdobi,pozice,track/album,interpret,vydavatel,soubor,youtube


In [10]:
df.loc[df['soubor'].str.contains('2503'),'obdobi']="8. týden 2018"

In [11]:
df[df['obdobi'] == '5. týden 2022'][['hitparada','soubor']].drop_duplicates()

,hitparada,soubor
64350,CZ - ALBUMS - TOP 100,14-2755.html
70550,CZ - ALBUMS - TOP 100,14-2819.html
161950,CZ - RADIO - TOP 100,2-2755.html
168150,CZ - RADIO - TOP 100,2-2819.html
189720,CZ - RADIO - TOP 20 MODERN ROCK,21-2755.html
190958,CZ - RADIO - TOP 20 MODERN ROCK,21-2819.html
200149,CZ - ALBUMS CLASSICAL - TOP 10,25-2755.html
200769,CZ - ALBUMS CLASSICAL - TOP 10,25-2819.html
244989,CZ - SINGLES DIGITAL - TOP 100,30-2755.html
251189,CZ - SINGLES DIGITAL - TOP 100,30-2819.html


In [12]:
df[df['soubor'].str.contains('2819')].groupby('obdobi').size()

obdobi
5. týden 2022    830
dtype: int64

In [13]:
df[df['obdobi'] == '5. týden 2023']

,Unnamed: 0,hitparada,obdobi,pozice,track/album,interpret,vydavatel,soubor,youtube


In [14]:
df.loc[df['soubor'].str.contains('2819'),'obdobi']="5. týden 2023"

In [15]:
df.groupby('obdobi')['track/album'].nunique().nlargest(10)

obdobi
17. týden 2019    547
20. týden 2019    541
29. týden 2019    541
46. týden 2017    540
28. týden 2019    539
41. týden 2017    538
45. týden 2017    538
20. týden 2017    537
26. týden 2018    537
32. týden 2018    537
Name: track/album, dtype: int64

In [16]:
df = df.drop(columns=['soubor'])

## Opravy a čištění

In [17]:
hitparady = df['hitparada'].drop_duplicates().to_list()
hitparady

['CZ - ALBUMS - TOP 100',
 'CZ - RADIO - TOP 100',
 'CZ - RADIO - TOP 20 MODERN ROCK',
 'CZ - ALBUMS CLASSICAL - TOP 10',
 'CZ - SINGLES DIGITAL - TOP 100',
 'CZ - SINGLES DIGITAL - TOP 50  CZ',
 'SK - ALBUMS - TOP 100',
 'SK - RADIO - TOP 100',
 'SK - RADIO - TOP 50 SK',
 'SK - SINGLES DIGITAL - TOP 100',
 'SK - SINGLES DIGITAL - TOP 50 SK',
 'CZ - RADIO - TOP 50 CZ']

### Aby byla data strojově čitelná

In [18]:
df['tyden'] = df['obdobi'].apply(lambda x: x.split(". t")[0] if ". t" in x else None)

In [19]:
df['tyden'] = df['tyden'].apply(lambda x: f"0{x}" if x and len(x) == 1 else x)

In [20]:
df['mesic'] = df['obdobi'].apply(lambda x: x.split("/")[0].split(" ")[1] if "Měsíc" in x else None)

In [21]:
df['mesic'] = df['mesic'].apply(lambda x: f"0{x}" if x and len(x) == 1 else x)

In [22]:
df['rok'] = df['obdobi'].apply(lambda x: x[-4:])

In [23]:
df.groupby('tyden')['track/album'].nunique().nsmallest(15)

tyden
5152       188
200903     190
200809     198
50.+51     206
53         328
52.+53     487
27.+28     519
30.+31     519
52.+01     524
52        1036
51        1785
51.+52    2072
50        4002
01        4108
31        4108
Name: track/album, dtype: int64

### Oprava chybných dat

In [24]:
df["rok"] = df["rok"].apply(lambda x: x.replace("223","2023").replace(" 201","2010").replace("2104","2014").strip())

### Týdny dvouciferně

In [25]:
df['tyden'].drop_duplicates().to_list()

['01',
 '02',
 '05',
 '06',
 '07',
 '08',
 '09',
 '10',
 '11',
 '12',
 '31',
 '32',
 '33',
 '34',
 '35',
 '36',
 '37',
 '38',
 '39',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '19',
 '20',
 '21',
 '22',
 '23',
 '24',
 '25',
 '26',
 '28',
 '27',
 '29',
 '30',
 None,
 '40',
 '41',
 '42',
 '43',
 '44',
 '45',
 '46',
 '47',
 '48',
 '49',
 '50',
 '5152',
 '03',
 '04',
 '51',
 '200809',
 '52',
 '200903',
 '51.+52',
 '50.+51',
 '53',
 '30.+31',
 '52.+01',
 '52.+53',
 '27.+28']

In [26]:
def cistetydny(x):
    
    try:
        
        x = x.replace(".","").replace("+","")
        if len(x) == 2:
            t = x
        else:
            if x[0:2] != "52":
                t = x[-2:]
            else:
                t = "52"
    
    except:
        
        t = x
    
    return t

In [27]:
cistetydny('200809')

'09'

In [28]:
df['tyden'] = df['tyden'].apply(lambda x: cistetydny(x))

In [29]:
df['tyden'].drop_duplicates().to_list()

['01',
 '02',
 '05',
 '06',
 '07',
 '08',
 '09',
 '10',
 '11',
 '12',
 '31',
 '32',
 '33',
 '34',
 '35',
 '36',
 '37',
 '38',
 '39',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '19',
 '20',
 '21',
 '22',
 '23',
 '24',
 '25',
 '26',
 '28',
 '27',
 '29',
 '30',
 None,
 '40',
 '41',
 '42',
 '43',
 '44',
 '45',
 '46',
 '47',
 '48',
 '49',
 '50',
 '52',
 '03',
 '04',
 '51',
 '53']

### Doplnění týdne

In [30]:
def weekuj(rok, tyden):
    try:
        return f"""{rok}-{tyden:02}"""
    except:
        return None

In [31]:
df['week'] = df.apply(lambda x: weekuj(x['rok'], x['tyden']), axis = 1)

In [32]:
df['week']

0         2006-01
1         2006-01
2         2006-01
3         2006-01
4         2006-01
           ...   
504271    2023-47
504272    2023-47
504273    2023-47
504274    2023-47
504275    2023-47
Name: week, Length: 504276, dtype: object

### Pryč s dvojitými mezerami

In [33]:
df = df.apply(lambda x:x.replace("  "," "))

### Pryč s velkými písmeny (nebo jejich většinou)

In [34]:
slovodelice = [" ",".","/"]
for s in slovodelice:
    df['interpret'] = df['interpret'].apply(lambda x: s.join(word.title() for word in str(x).split(s)))
    df['vydavatel'] = df['vydavatel'].apply(lambda x: s.join(word.title() for word in str(x).split(s)))

In [35]:
df.sample(5)

,Unnamed: 0,hitparada,obdobi,pozice,track/album,interpret,vydavatel,youtube,tyden,mesic,rok,week
82449,99,CZ - RADIO - TOP 100,15. týden 2007,100,Crazy,Alanis Morissette,Warner Music,NaN,15,None,2007,2007-15
229342,53,CZ - SINGLES DIGITAL - TOP 100,32. týden 2019,54,Shape of You,Ed Sheeran,Warner Music,https://www.youtube.com/watch?v=_dK2tDK9grQ,32,None,2019,2019-32
2247,47,CZ - ALBUMS - TOP 100,46. týden 2006,48,Live,Hradišťan,Ir,NaN,46,None,2006,2006-46
196759,3,CZ - ALBUMS CLASSICAL - TOP 10,15. týden 2016,4,Best Of Jonas Kaufmann,Různí,Universal Music,NaN,15,None,2016,2016-15
13815,15,CZ - ALBUMS - TOP 100,21. týden 2011,16,Duety nabílo +DVD,Lucie Bílá,Warner Music,NaN,21,None,2011,2011-21


### Sjednocení jmen

In [36]:
df[df['interpret'].str.lower().str.contains("vířat",na=False)]['interpret'].drop_duplicates().to_list()

['Sto Zvířat']

In [37]:
spravna_jmena = {
    "Ondřej G. Brzobohatý": "Ondřej Brzobohatý",
    "M. Cyrus": "Miley Cyrus",
    "Ulrychovi Hana & Petr": "Hana A Petr Ulrychovi",
    "Ulrychovci":  "Hana A Petr Ulrychovi",
    "Miroslav Žbirka": "Miro Žbirka",
    "P.Hapka": "Petr Hapka",
    "Natalie Grossová": "Natálie Grossová",
    "Tichá Kateřina Marie": "Kateřina Marie Tichá",
    "Al Dhaba Gába": "Gabriela Al Dhábba",
    "Křížek Láďa": "Ladislav Křížek",
    "AC/DC": "ACDC",
    " A Jiní": "",
    " A ": " a "
}

In [38]:
interpreti = df['interpret'].drop_duplicates().to_list()
[x for x in interpreti if len(x.split()) > 1 and f"{x.split()[1]} {x.split()[0]}" in interpreti]

['Nedvěd František',
 'Simon Paul',
 'Jones Norah',
 'Baumaxa Xavier',
 'Manu Chao',
 'Gahan  Dave',
 'Lipa Peter',
 'Duran Duran',
 'Sodoma Viktor',
 'Pospíšil Luboš & 5P',
 'Ward Shayne',
 'Toxxx Hugo',
 'Tobias Szidi',
 'Cohen Avishai',
 'Petřina Ota',
 'Janů Petra',
 'Pop Iggy',
 'Hrůza Michal & Ready Kirken',
 'Streisand Barbra',
 'Satriani Joe',
 'Bárta Dan & Robert Balzar Trio',
 'Knechtová Katarína & Peha',
 'Absolonová Monika',
 'Hybš Václav',
 'Spálený Jan',
 'Šporcl Pavel & Romano Stilo',
 'Ruml Ondřej',
 'Merta Vladimír & Etc',
 'Janečková Patrícia',
 'Havelka Ondřej',
 'Dragoun Roman',
 'Smith  Patti',
 'Young Neil & Crazy Horse',
 'Brown Chris',
 'Mercury Freddie & Caballé Monserrat',
 'Horáček Michal & Lucia Šoralová',
 'Havelka Ondřej A Jeho Melody Makers',
 'Merta Vladimír & Dobrá Úroda',
 'Lang Lang',
 'Dědeček Jiří',
 'Bruni Carla',
 'Mišík Vladimír & Kovářík Mirek & Čdg',
 'Bonamassa Joe & Beth Hart',
 'Zagorová Hana / Petr Rezek',
 'Müller Richard & Fragile',
 'Pet

In [39]:
obracena_jmena = ["Bohouš Josef","Bílá Lucie","Žbirka Miro","Vondráčková Lucie", "Jiřina Anna Jandová", "Smolík Jakub", "Sofian Medjmedj", "Josef Bohouš", "Müller Richard","Rolins Dara", "Zmožek Marcel", "Hrůza Michal", "Deyl David", "Bende Petr", "Zonyga Jiří", "Hapka Petr", "Nedvěd František", "Mrózek Daniel", "Issa Sámer","Koller David","Ondra Jakub", "Smejkal Jaro", "Vondráčková Helena", "Střihavka Kamil", "Charvátová Jitka", "Goldová Gabriela","Přenosilová Yvonne", "Machálková Leona","Prokop Michal","Suchý Jiří","Langerová Aneta","Daniel Šafařík", "Jandová Marta", "Horváth Vlastimil", "Horecká Hana", "Farna Ewa", "John Wolfhooker", "Adam Janota",  "Dyk Vojtěch", "Jíšová Pavlína", "Sanchez Yvonne", "Ruppert Matěj", "Burian Michael", "Brzobohatá Katka", "Václav Rouček", "Šinkorová Daniela", "Šeps Michal", "Absolonová Monika", "Zíma Josef", "Rouček Václav","Schmitzer Jiří","Ztracený Marek", "Zemanová Bára", "Bastien Alan", "Maxa Martin", "Matuš Bohuš", "Hron Vláďa", "Hudček Michal", "Háma Aleš", "Hübsch Kamila", "Hůlka Daniel", "Milfajtová Sára", "Krajčo Richard", "Kraus David", "Kreyson Memorial", "Krézlová Karolína", "Josef Vágner, Charlotte Gottová", "Hladík Radim", "Peter Fider", "Drda Zbyněk", "Paul Project", "Povýšilová Monika", "Poláček Petr", "Poliak Michal", "Hanák Tomáš", "Hayek Martin", "Nekonečný Dan", "Neckář Václav", "Novotný Miloš","Rosová Sandra","Vojtek Roman","Bagová Ivanna","Kerndlová Tereza","Kotvald Petr",'Simon Paul',"Albarn Damon",
 'Jones Norah',
 'Baumaxa Xavier',
 'Chao Manu',
 'Gahan Dave',
 'Lipa Peter',
 'Pospíšil Luboš',
 'Sodoma Viktor',
 'Ward Shayne',
 'Toxxx Hugo',
 'Tobias Szidi',
 'Cohen Avishai',
 'Petřina Ota',
 'Janů Petra',
 'Pop Iggy',
 'Streisand Barbra',
 'Satriani Joe',
 'Bárta Dan',
 'Knechtová Katarína',
 'Hybš Václav',
 'Spálený Jan',
 'Šporcl Pavel',
 'Ruml Ondřej',
 'Merta Vladimír',
 'Havelka Ondřej',
 'Dragoun Roman',
 'Smith Patti',
 'Young Neil',
 'Brown Chris',
 'Mercury Freddie',
 'Caballé Monserrat',
 'Horáček Michal',
 'Havelka Ondřej',
 'Merta Vladimír',
 'Dědeček Jiří',
 'Bruni Carla',
 'Mišík Vladimír',
 'Bonamassa Joe',
 'Zagorová Hana',
 'Petty Tom',
 'Bennett Tony',
 'Dusilová Lenka',
 'Ondřej Havelka',
 'Ruml Ondřej',
 'Szidi Tobias',
 'Ondřej Havelka',
 'Roman Dragoun And His Angels',
 'Avishai Cohen Trio',
 'Sinatra Frank',
 'Zdeněk Bína',
 'Melody Gardot',
 'Petřina Ota',
 'Cooper Alice',
 'Satriani Joe',
 'Veverka Vilém',
 'Bernstein Leonard',
 'Dragoun Roman',
 'Clyro Biffy',
 'Gardot Melody',
 'Spálený Jan',
 'Hybš Václav',
 'Michal Hromek',
 'Bernstein Leonard',
 'Veverka Vilém',
 'Svěcený Jaroslav',
 'Semerádová Jana',
 'Ramazzotti Eros & Anastacia',
 'Foret Michael',
 'Ward Shayne',
 'Keys Alicia & White Jack',
 'Burke Alexandra Feat. Flo Rida',
 'Bieber Justin',
 'Tempah Tinie',
 'Williams Robbie',
 'Iglesias Enrique',
 'Solveig Martin',
 'Lopez Jennifer',
 'Sinclar Bob',
 'Posner Mike',
 'Brown Chris',
 'Solveig Martin',
 'Lopez Jennifer',
 'Brown Chris',
 'Gomez Selena',
 'Murs Olly',
 'Harris Calvin',
 'Burke Alexandra',
 'Konvičková Markéta',
 'Harris Calvin',
 'Grey Skylar',
 'Romero Nicky',
 'Solveig Martin',
 'Derulo Jason',
 'Tempah Tinie',
 'Lavigne Avril',
 'Harris Calvin',
 'Bublé Michael',
 'Aristone Peter',
 'Garand Paulie',
 'Azalea Iggy',
 'Paul Sean',
 'Ponte Gabry',
 'Cooper Alice',
 'Bína Zdeněk',
 'Bezdeda Tomáš',
 'Opatovský Robo',
 'Kerndlová Tereza',
 'Kmeťo Igor',
 'Lipa Peter',
 'Tomáš Bezdeda',
 'Mirgová Dominika',
                 'Gahan Dave',
                 'Cohen Avishai',
                 'Smith Patti',
                 'Dragoun Roman',
                 'Gott Karel',
                 'Savka Tomáš',
                 'Schmitzer Jiří',
                 'Cristovao Ben',
                 "Norisová Zuzana",
                 "Guetta David",
                 "Gaudino Alex",
                 "Dione Aura",
                 "Wolf Laurent",
                 "Scherzinger Nicole",
                 "Kingston Sean",
                 "Hilson Keri",
                 "Ryan Kate",
                 "Wolf Karl"]

In [40]:
obracena_jmena

['Bohouš Josef',
 'Bílá Lucie',
 'Žbirka Miro',
 'Vondráčková Lucie',
 'Jiřina Anna Jandová',
 'Smolík Jakub',
 'Sofian Medjmedj',
 'Josef Bohouš',
 'Müller Richard',
 'Rolins Dara',
 'Zmožek Marcel',
 'Hrůza Michal',
 'Deyl David',
 'Bende Petr',
 'Zonyga Jiří',
 'Hapka Petr',
 'Nedvěd František',
 'Mrózek Daniel',
 'Issa Sámer',
 'Koller David',
 'Ondra Jakub',
 'Smejkal Jaro',
 'Vondráčková Helena',
 'Střihavka Kamil',
 'Charvátová Jitka',
 'Goldová Gabriela',
 'Přenosilová Yvonne',
 'Machálková Leona',
 'Prokop Michal',
 'Suchý Jiří',
 'Langerová Aneta',
 'Daniel Šafařík',
 'Jandová Marta',
 'Horváth Vlastimil',
 'Horecká Hana',
 'Farna Ewa',
 'John Wolfhooker',
 'Adam Janota',
 'Dyk Vojtěch',
 'Jíšová Pavlína',
 'Sanchez Yvonne',
 'Ruppert Matěj',
 'Burian Michael',
 'Brzobohatá Katka',
 'Václav Rouček',
 'Šinkorová Daniela',
 'Šeps Michal',
 'Absolonová Monika',
 'Zíma Josef',
 'Rouček Václav',
 'Schmitzer Jiří',
 'Ztracený Marek',
 'Zemanová Bára',
 'Bastien Alan',
 'Maxa Martin

In [41]:
for o in obracena_jmena:
    spravna_jmena[o] = f"{o.split()[1]} {o.split()[0]}"

In [42]:
for key, val in spravna_jmena.items():
    df['interpret'] = df['interpret'].apply(lambda x: x.replace(key,val))

In [43]:
interpreti = df['interpret'].drop_duplicates().to_list()
[x for x in interpreti if len(x.split()) > 1 and f"{x.split()[1]} {x.split()[0]}" in interpreti]

['Gahan  Dave',
 'Duran Duran',
 'Avishai Cohen',
 'Janečková Patrícia',
 'Roman Dragoun',
 'Smith  Patti',
 'Lang Lang',
 'Dragoun Roman',
 'Cohen Avishai',
 'Laurie Anderson & Kronos Quartet',
 'Jan Vodňanský a Haňa Navarová',
 'David Michal a Lucie Vondráčková',
 'Timberlake Justin & Jay-Z',
 'Junge Junge & Kyle Pearce',
 'Junge Junge',
 'Junge Junge & Joe Cleere',
 'Patrícia Janečková',
 'Loko Loko & Rytmus',
 'Fifty Fifty',
 'Loko Loko',
 'Keith Urban Feat. P!Nk',
 'Loko Loko & Momo',
 'Loko Loko & Hard Rico']

### Hlavní intepreti

In [44]:
df[df['interpret'].str.contains(" a ")]['interpret'].drop_duplicates()

86                  Jaroslav Uhlíř a Zdeněk Svěrák
390                          Hana a Petr Ulrychovi
874                                  Mňága a Žďorp
9590                          Ryvolovi Wabi a Miky
12990                               Žalman a Spol.
14050                 Robert Křesťan a Druhá Tráva
17784          Havelka Ondřej a Jeho Melody Makers
21084                      Michnová Zuzana a Hosté
23460                   Wabi Daněk a Ďáblovo Stádo
24147                  Petr Hapka a Michal Horáček
28567                  Věra Špinarová a Petra Janů
30456                           Laura a Její Tygři
34097                     Michal Prokop a Framus 5
58233                       Y.Simonová a M.Chladil
60492                Jan Vodňanský a Haňa Navarová
77004                             System Of a Down
88942                                Marek a Kamil
105410            David Michal a Lucie Vondráčková
109047                   Markéta Konvičková a Elis
111610    Eddie Stoilow Feat. E

In [45]:
def kdojeprvni(x):
    delice = ["&","/","(feat.","feat.","Feat."," a "]
    for d in delice:
        if d in x:
            x = x.split(d)[0]
    return x.strip()

In [46]:
df['interpret_hlavni'] = df['interpret'].apply(lambda x: kdojeprvni(x))

In [47]:
df['interpret_hlavni'].drop_duplicates().to_list()

['Petr Kolář',
 'Chinaski',
 'No Name',
 'Nedvědi',
 'Pavol Habera',
 'Aneta Langerová',
 'Těžkej Pokondr',
 'Robbie Williams',
 'Damichi',
 'Divokej Bill',
 'Petr Novák',
 'Různí',
 'Vlasta Horváth',
 'Rammstein',
 'Madonna',
 'Petr Bende',
 'Helena Zeťová',
 'Jakub Smolík',
 'Čechomor',
 'Richard Müller',
 'Tatu',
 'Karel Gott',
 'Depeche Mode',
 'Petr Muk',
 'Lenka Filipová',
 'Michal David',
 'Black Eyed Peas',
 'Anastacia',
 'Juanes',
 'Amiri Ali',
 'Sámer Issa',
 'Tři Sestry',
 'Věra Špinarová',
 'Enya',
 'Karel Kryl',
 'Maxim Turbulenc',
 'Miro Žbirka',
 'Zuzana Navarová',
 'Iveta Bartošová',
 'Marie Rottrová',
 'Pražský Výběr',
 'Hana Hegerová',
 'Dan Bárta',
 'Iva Frühlingová',
 'Jiří Schelinger',
 'Leoš Mareš',
 'Janek Ledecký',
 'Eminem',
 'Jaroslav Uhlíř',
 'Petr Hapka',
 'Lucie Vondráčková',
 'Alanis Morissette',
 '4Tet',
 'Jana Kirschner',
 'James Blunt',
 'Crazy Frog',
 'Lenka Dusilová',
 'Victoria',
 'Hana Zagorová',
 'Marika Gombitová',
 'Radůza',
 '2M',
 'Peha',
 'Mic

In [48]:
len(df[(df['hitparada'] == 'CZ - RADIO - TOP 100')]['interpret_hlavni'].drop_duplicates().to_list())

1440

### Album/track

In [49]:
df['album'] = df.apply(lambda row: row['track/album'] if 'ALBUM' in row['hitparada'] else None, axis = 1)

In [50]:
df['track'] = df.apply(lambda row: row['track/album'] if 'ALBUM' not in row['hitparada'] else None, axis = 1)

In [51]:
df = df.drop(columns=['track/album','obdobi'])

In [52]:
poradi = ['hitparada','rok','mesic','tyden','week','pozice','album','track','interpret','interpret_hlavni','vydavatel','youtube']

In [53]:
df = df.reindex(columns=poradi)

In [54]:
df[(df['week'] == '2017-08') & (df['hitparada'].str.contains('RADIO'))].shape

(318, 12)

In [55]:
df[(df['week'] == '2017-08') & (df['hitparada'].str.contains('RADIO'))].head(60)

,hitparada,rok,mesic,tyden,week,pozice,album,track,interpret,interpret_hlavni,vydavatel,youtube
132750,CZ - RADIO - TOP 100,2017,None,08,2017-08,1,None,Shape of You,Ed Sheeran,Ed Sheeran,Warner Music,https://www.youtube.com/watch?v=_dK2tDK9grQ
132751,CZ - RADIO - TOP 100,2017,None,08,2017-08,2,None,Human,Rag 'N' Bone Man,Rag 'N' Bone Man,Sony Music,https://www.youtube.com/watch?v=L3wKzyIN1yk
132752,CZ - RADIO - TOP 100,2017,None,08,2017-08,3,None,Scars To Your Beautiful,Alessia Cara,Alessia Cara,Universal Music,https://www.youtube.com/watch?v=MWASeaYuHZo
132753,CZ - RADIO - TOP 100,2017,None,08,2017-08,4,None,Není nám do pláče,Chinaski,Chinaski,Brainzone,NaN
132754,CZ - RADIO - TOP 100,2017,None,08,2017-08,5,None,Say You Won't Let Go,James Arthur,James Arthur,Sony Music,https://www.youtube.com/watch?v=0yW7w8F2TVA
132755,CZ - RADIO - TOP 100,2017,None,08,2017-08,6,None,If I Were Sorry,Frans,Frans,Sony Music,https://www.youtube.com/watch?v=4emC0VDQGz8
132756,CZ - RADIO - TOP 100,2017,None,08,2017-08,7,None,Only One,Sigala & Digital Farm Animals,Sigala,Sony Music,https://www.youtube.com/watch?v=ug0A0uuSpDE
132757,CZ - RADIO - TOP 100,2017,None,08,2017-08,8,None,Would I Lie To You,David Guetta & Chris Willis,David Guetta,Warner Music,https://www.youtube.com/watch?v=u0pmV7GoTjchtt...
132758,CZ - RADIO - TOP 100,2017,None,08,2017-08,9,None,Lost On You,Lp,Lp,Universal Music,https://www.youtube.com/watch?v=hn3wJ1_1Zsg
132759,CZ - RADIO - TOP 100,2017,None,08,2017-08,10,None,Run Run Run,Junge Junge & Kyle Pearce,Junge Junge,Universal Music,https://www.youtube.com/watch?v=nme9ph8qku4


### Seřazení

In [56]:
df = df.sort_values(by=['hitparada','rok','mesic','tyden','pozice'])

In [57]:
df.sample(10)

,hitparada,rok,mesic,tyden,week,pozice,album,track,interpret,interpret_hlavni,vydavatel,youtube
204524,CZ - SINGLES DIGITAL - TOP 100,2015,None,02,2015-02,36,None,Chandlier,Sia,Sia,Sony Music,http://www.youtube.com/watch?v=2vjPBrBU-TM
417551,SK - SINGLES DIGITAL - TOP 100,2021,None,40,2021-40,2,None,Hey G! (feat. Separ),Gleb,Gleb,Universal Music,https://www.youtube.com/watch?v=XrrZ7ds9oqY
160577,CZ - RADIO - TOP 100,2021,None,45,2021-45,28,None,EveryTime I Cry,Ava Max,Ava Max,Warner Music,https://www.youtube.com/watch?v=CFMz9DOhaJ8
142860,CZ - RADIO - TOP 100,2019,None,04,2019-04,11,None,Flames,David Guetta & Sia,David Guetta,Warner Music,NaN
246120,CZ - SINGLES DIGITAL - TOP 100,2022,None,14,2022-14,32,None,Mám tě v p...!!,Šafařík Daniel,Šafařík Daniel,Records Dk,https://www.youtube.com/watch?v=BSg7NZNAAWw
296059,SK - ALBUMS - TOP 100,2019,05,None,None,71,Pesničky pre (ne)poslušné deti,None,Miro Jaroš,Miro Jaroš,Galgan Music,NaN
483929,CZ - RADIO - TOP 50 CZ,2017,None,22,2017-22,37,None,Život,No Name,No Name,Brainzone,https://www.youtube.com/watch?v=Ssc5Iajb_xQ
111960,CZ - RADIO - TOP 100,2013,None,04,2013-04,11,None,Candy,Robbie Williams,Robbie Williams,Universal Music,NaN
165741,CZ - RADIO - TOP 100,2022,None,36,2022-36,92,None,Kings & Queens,Ava Max,Ava Max,Warner Music,NaN
68060,CZ - ALBUMS - TOP 100,2022,None,36,2022-36,11,Mercury - Act 1,None,Imagine Dragons,Imagine Dragons,Universal Music,https://www.youtube.com/playlist?list=PLxA687t...


## Oprava chybných oprav (nutno zde)

In [58]:
df.loc[df['interpret'] == 'Nico & Vinz',"interpret_hlavni"]="Nico & Vinz"
df.loc[df['interpret'] == 'Zagorová',"interpret_hlavni"]="Hana Zagorová"

## Youtube do samostatného dataframu

In [59]:
import os

if not os.path.exists("data"):
    os.mkdir("data")
else:
    pass

In [60]:
df

,hitparada,rok,mesic,tyden,week,pozice,album,track,interpret,interpret_hlavni,vydavatel,youtube
1850,CZ - ALBUMS - TOP 100,2006,05,None,None,1,Divokej Bill,None,Divokej Bill,Divokej Bill,Warner Music,NaN
1851,CZ - ALBUMS - TOP 100,2006,05,None,None,2,Stadium Arcadium,None,Red Hot Chili Peppers,Red Hot Chili Peppers,Warner Music,NaN
1852,CZ - ALBUMS - TOP 100,2006,05,None,None,3,Asi se něco děje,None,Ready Kirken,Ready Kirken,Universal Music,NaN
1853,CZ - ALBUMS - TOP 100,2006,05,None,None,4,Pearl Jam,None,Pearl Jam,Pearl Jam,Sony Music,NaN
1854,CZ - ALBUMS - TOP 100,2006,05,None,None,5,Schrei,None,Tokio Hotel,Tokio Hotel,Universal Music,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
452391,SK - SINGLES DIGITAL - TOP 50 SK,2023,None,47,2023-47,46,None,Hey G! (feat. Separ),Gleb,Gleb,Universal Music,https://www.youtube.com/watch?v=XrrZ7ds9oqY
452392,SK - SINGLES DIGITAL - TOP 50 SK,2023,None,47,2023-47,47,None,New York Freestyle,Separ,Separ,Ruka Hore,https://www.youtube.com/watch?v=Q0BJcbr3D-M
452393,SK - SINGLES DIGITAL - TOP 50 SK,2023,None,47,2023-47,48,None,Niekde v oblakoch,Saul,Saul,Fck Them/Universal Music,NaN
452394,SK - SINGLES DIGITAL - TOP 50 SK,2023,None,47,2023-47,49,None,SPIT IN MY FACE!,Thxsomch,Thxsomch,Warner Music,https://www.youtube.com/watch?v=ZtCOsQqrBrs


In [61]:
yt = df[['interpret_hlavni','track','youtube']].drop_duplicates()
yt.to_csv(os.path.join("data","youtube_linky.csv"))

In [62]:
df = df.drop(columns=['youtube'])

## Export do jednotlivých dataframů

In [63]:
df

,hitparada,rok,mesic,tyden,week,pozice,album,track,interpret,interpret_hlavni,vydavatel
1850,CZ - ALBUMS - TOP 100,2006,05,None,None,1,Divokej Bill,None,Divokej Bill,Divokej Bill,Warner Music
1851,CZ - ALBUMS - TOP 100,2006,05,None,None,2,Stadium Arcadium,None,Red Hot Chili Peppers,Red Hot Chili Peppers,Warner Music
1852,CZ - ALBUMS - TOP 100,2006,05,None,None,3,Asi se něco děje,None,Ready Kirken,Ready Kirken,Universal Music
1853,CZ - ALBUMS - TOP 100,2006,05,None,None,4,Pearl Jam,None,Pearl Jam,Pearl Jam,Sony Music
1854,CZ - ALBUMS - TOP 100,2006,05,None,None,5,Schrei,None,Tokio Hotel,Tokio Hotel,Universal Music
...,...,...,...,...,...,...,...,...,...,...,...
452391,SK - SINGLES DIGITAL - TOP 50 SK,2023,None,47,2023-47,46,None,Hey G! (feat. Separ),Gleb,Gleb,Universal Music
452392,SK - SINGLES DIGITAL - TOP 50 SK,2023,None,47,2023-47,47,None,New York Freestyle,Separ,Separ,Ruka Hore
452393,SK - SINGLES DIGITAL - TOP 50 SK,2023,None,47,2023-47,48,None,Niekde v oblakoch,Saul,Saul,Fck Them/Universal Music
452394,SK - SINGLES DIGITAL - TOP 50 SK,2023,None,47,2023-47,49,None,SPIT IN MY FACE!,Thxsomch,Thxsomch,Warner Music


In [64]:
for h in hitparady:
    df_p = df[df["hitparada"] == h].reset_index(drop=True)
    if "ALBUMS" in h:
        df_p = df_p.drop(columns=['track'])
    else:
        df_p = df_p.drop(columns=['album'])
    df_p2 = df_p[~df_p['mesic'].isnull()]
    h = h.replace(" ","_").replace("-_","")
    if len(df_p2) > 0:
        df_p2 = df_p2.drop(columns=['tyden'])
        df_p2.to_csv(os.path.join("data",f"{h}_mesicni.csv"), index=False)
    df_p2 = df_p[~df_p['tyden'].isnull()]
    if len(df_p2) > 0:
        df_p2 = df_p2.drop(columns=['mesic'])
        df_p2.to_csv(os.path.join("data",f"{h}_tydenni.csv"), index=False)